In [1]:
import requests
from pprint import pprint
import csv
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import time
from tqdm import tqdm
import calendar
import os
import string
import random
import boto3


def get_stockprice(company_symbol: str = 'MSFT'):
    endpoint = "https://www.alphavantage.co/query"
    parameters = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": company_symbol,
        "outputsize": 'full'
    }
    for _ in range(100):
        parameters['apikey'] = ''.join(random.choices(string.ascii_uppercase + string.digits, k=15))
        # Send a GET request to the API endpoint
        response = requests.get(endpoint, params=parameters)
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            if 'Note' not in data: 
                break
            print(f'API key {parameters["apikey"]} has been used too many times. response note: {data["Note"]}')
            data = None
            time.sleep(1)
        else: 
            print(f'API key {parameters["apikey"]} has returned an error. response note: {response.json()}')
    return data


# get stock price for all tickers
def get_stockprice_all(stocks_to_watch: list):
    os.makedirs('prices', exist_ok=True)
    # only get stock price for stocks that are not in the directory
    seen_stocks = [f.split('.')[0] for f in os.listdir('prices') if os.path.isfile(os.path.join('prices', f))]
    for ticker in tqdm([t for t in stocks_to_watch if t not in seen_stocks]):
        data = get_stockprice(ticker)
        if data is None: 
            print(f'Unnable to fetch data for {ticker}')
            continue
        with open(f'prices/{ticker}.json', 'w') as outfile:
            json.dump(data, outfile, indent=4)

In [16]:
# define a boto resource in the ohio region
dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
table = dynamodb.Table('StockSentiment')
# get a list of all items in the Stock column sorted by frequency
sentiment_ticker_list = pd.DataFrame(table.scan()['Items'])
# convert Date column to datetime
sentiment_ticker_list['Date'] = pd.to_datetime(sentiment_ticker_list['Date'])
# make the index the Date column
sentiment_ticker_list.set_index('Date', inplace=True)
sentiment_ticker_list.sort_index(inplace=True, ascending=False)
sentiment_ticker_list

,Date,ticker_sentiment_score,ticker_sentiment_label,Stock,source,url,relevance_score
0,20230224T125626,-0.134091,Neutral,AKBA,Benzinga,https://www.benzinga.com/news/23/02/31046036/w...,0.130217
1,20230221T120000,0.339446,Somewhat-Bullish,TMXXF,Canada Newswire,https://www.newswire.ca/news-releases/internat...,0.318897
2,20230224T210000,0.395279,Bullish,TMXXF,Canada Newswire,https://www.newswire.ca/news-releases/tmx-grou...,0.896158
3,20230303T000028,0.108872,Neutral,FOX,Motley Fool,https://www.fool.com/earnings/call-transcripts...,0.005401
4,20230306T212603,-0.118802,Neutral,FOX,CNBC,https://www.cnbc.com/2023/03/06/new-bipartisan...,0.07031
...,...,...,...,...,...,...,...
4143,20230301T193004,0.237186,Somewhat-Bullish,RETA,Benzinga,https://www.benzinga.com/news/earnings/23/03/3...,0.119235
4144,20230302T215509,0.182761,Somewhat-Bullish,RETA,Benzinga,https://www.benzinga.com/news/23/03/31178455/b...,0.142565
4145,20230303T165500,0.050318,Neutral,RETA,Investors Business Daily,https://www.investors.com/news/stock-market-ra...,0.038468
4146,20230302T172419,0.212924,Somewhat-Bullish,XELA,Benzinga,https://www.benzinga.com/news/earnings/23/03/3...,0.129516


In [17]:
# get a list of tickers sorted by frequency
sorted_tickers = sentiment_ticker_list['Stock'].value_counts().index.tolist()
sorted_tickers

['FOREX:USD',
 'META',
 'GS',
 'TSLA',
 'GOOG',
 'JPM',
 'MS',
 'CRYPTO:BTC',
 'C',
 'BRK-A',
 'MSFT',
 'NVDA',
 'BCS',
 'BAC',
 'WFC',
 'CRYPTO:ETH',
 'COIN',
 'WMT',
 'BABA',
 'AAPL',
 'AWON',
 'CURN',
 'SZIHF',
 'BLK',
 'FOREX:EUR',
 'AMZN',
 'DSSMY',
 'BYND',
 'EVR',
 'NFLX',
 'RCKY',
 'HD',
 'CRYPTO:XCP',
 'COST',
 'PFE',
 'KO',
 'MCO',
 'TOL',
 'FOREX:JPY',
 'APG',
 'STT',
 'WBD',
 'SPXCF',
 'ABNB',
 'ZS',
 'MRNA',
 'ETSY',
 'LI',
 'NYT',
 'FSLR',
 'TCTZF',
 'GM',
 'CRYPTO:DOGE',
 'AXP',
 'NIO',
 'FOREX:GBP',
 'PYPL',
 'M',
 'BIDU',
 'RIVN',
 'PANW',
 'LDNXF',
 'ROKU',
 'AVGO',
 'FOREX:CAD',
 'JNJ',
 'XPEV',
 'CCORF',
 'FOREX:UYU',
 'OXY',
 'SQ',
 'SSNLF',
 'RYDAF',
 'AMD',
 'AMAT',
 'MDNDF',
 'BBY',
 'TTD',
 'TGT',
 'MELI',
 'SI-P-A',
 'MA',
 'ABBV',
 'SCHW',
 'MORN',
 'SBUX',
 'BKNG',
 'WING',
 'NTES',
 'Z',
 'ENVX',
 'XOM',
 'QCOM',
 'PARA',
 'FOREX:CNY',
 'ON',
 'SHOP',
 'UPST',
 'GLP',
 'PSHG',
 'BA',
 'NVAX',
 'ZM',
 'BR',
 'RBCPF',
 'EBAY',
 'O',
 'CSCO',
 'GE',
 'MGM',
 '